In [2]:
!pip3 install wandb

     |████████████████████████████████| 1.7 MB 4.5 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 180 kB 47.2 MB/s 
     |████████████████████████████████| 139 kB 50.0 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=9406010e15008276456f8afd22416ba1de3649d80b3d6dbad54fa2a7587dcca5
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=1c2e721305760fc662478c1de32c3d1673fb6c861e4580925abef6c1c3248971
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [3]:
import cv2
import os
import wandb
import numpy as np
import cv2
import tensorflow as tf
from wandb.keras import WandbCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam

In [4]:
folder_address = '/content/drive/MyDrive/Persian_Recognition/'

In [ ]:
wandb.init(project='Persian_Recognition')

In [8]:
config = wandb.config
config.learning_rate = 0.001

In [9]:
width = 224
height = 224

In [10]:
batch_size = 64

In [11]:
data_generator = ImageDataGenerator(
    rescale = 1 / 255,
    validation_split = 0.2,
    horizontal_flip = True
)
train_data = data_generator.flow_from_directory(folder_address,
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True,
                                                    subset ='training')

val_data = data_generator.flow_from_directory(folder_address,
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True,
                                                    subset ='validation')

Found 2310 images belonging to 2 classes.
Found 576 images belonging to 2 classes.


In [12]:
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(width, height, 3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'max'
    )

94683136/94668760 [==============================] - 1s 0us/step


In [ ]:
base_model.summary()

In [14]:
for layer in base_model.layers[:-8]:
  layer.trainable = False

In [16]:
model = tf.keras.models.Sequential([
    base_model,
    Dense(2, activation='softmax')                          
])

In [17]:
model.compile(optimizer = tf.keras.optimizers.SGD(),
              loss = tf.keras.losses.binary_crossentropy,
              metrics='accuracy')

In [18]:
model.fit(train_data,
          steps_per_epoch = train_data.samples / batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples / batch_size,
          epochs = 12,
          class_weight = {0:1, 1:3},
          callbacks = [WandbCallback()])

Epoch 1/12
36/36 [==============================] - 1663s 45s/step - loss: 1.4825 - accuracy: 0.8792 - val_loss: 2.6810 - val_accuracy: 0.6406


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/12
36/36 [==============================] - 28s 784ms/step - loss: 0.2449 - accuracy: 0.9524 - val_loss: 0.5112 - val_accuracy: 0.8872
Epoch 3/12
36/36 [==============================] - 28s 779ms/step - loss: 0.1191 - accuracy: 0.9753 - val_loss: 0.4183 - val_accuracy: 0.9010
Epoch 4/12
36/36 [==============================] - 28s 795ms/step - loss: 0.1424 - accuracy: 0.9719 - val_loss: 0.3495 - val_accuracy: 0.8976
Epoch 5/12
36/36 [==============================] - 28s 774ms/step - loss: 0.0626 - accuracy: 0.9887 - val_loss: 0.3061 - val_accuracy: 0.9062
Epoch 6/12
36/36 [==============================] - 28s 772ms/step - loss: 0.0513 - accuracy: 0.9905 - val_loss: 0.3216 - val_accuracy: 0.9062
Epoch 7/12
36/36 [==============================] - 28s 766ms/step - loss: 0.0314 - accuracy: 0.9926 - val_loss: 0.3563 - val_accuracy: 0.8993
Epoch 8/12
36/36 [==============================] - 28s 770ms/step - loss: 0.0236 - accuracy: 0.9965 - val_loss: 0.3018 - val_accuracy: 0.9028

In [ ]:
model.save('/content/model.h5')